In [ ]:
from scipy.stats import norm

In [ ]:
def ucb(post_mean,post_std, beta):
    # Upper confidence bound (UCB) acquisition function for maximisation
    # Beta controls the degree exploitation/exploration, the higher beta the more exploration occurs
    acquisition_function = post_mean + (beta * post_std)
    return acquisition_function

In [ ]:
def probability_improvement(y,post_mean,post_std, eta):
    # Probability of improvement (PI) acquisition function for maximisation
    # Eta improvement by at least eta, as eta increases more exploration occurs
    y_max = float(np.max(y))
    z = (post_mean - y_max - eta) / (post_std + 1e-12)
    acquisition_function = norm.cdf(z)
    return acquisition_function

In [ ]:
def expected_improvement(post_mean, post_std, y_train, eps=1e-9):
    # Expected improvement (EI) acquisition function for maximisation
    post_std = np.maximum(post_std, eps)
    best_y = y_train.max()

    improvement = post_mean - best_y
    z = improvement / post_std

    ei = improvement * norm.cdf(z) + post_std * norm.pdf(z)
    return ei

In [ ]:
def acquisition_function_results(post_mean, post_std, candidates_no_transformation,
                                 beta, y_transform, eps, eta):
    # Print output of acquisiton functions to suggest where to explore next
    
    # Tune beta to control the amount of exploration
    ucb_acquistion_function = ucb(post_mean, post_std, beta)
    
    expected_improvement_function = expected_improvement(post_mean, post_std, y_transform, eps)
    
    # Tune eta to control the amount of exploration
    pi_acquisition_function = probability_improvement(y_transform, post_mean, post_std, eta)
    
    # Find index of the candidate that maximises each acquisition function
    idx_ucb = np.argmax(ucb_acquistion_function)
    idx_ei = np.argmax(expected_improvement_function)
    idx_pi = np.argmax(pi_acquisition_function)
    
    # Find predicted mean and standard deviation for each suggested point
    mean_ucb, std_ucb = post_mean[idx_ucb], post_std[idx_ucb]
    mean_ei,  std_ei  = post_mean[idx_ei],  post_std[idx_ei]
    mean_pi,  std_pi  = post_mean[idx_pi],  post_std[idx_pi]
    
    # Find the candidate points that maximise each acquisition function
    x_next_ucb = candidates_no_transformation[idx_ucb]
    x_next_ei  = candidates_no_transformation[idx_ei]
    x_next_pi  = candidates_no_transformation[idx_pi]
    
    print(f"Next point to explore maximising UCB acqusition function for Standard GP:\n{x_next_ucb}\n\
    predicted mean: {mean_ucb:.3f}, predicted std: {std_ucb:.3f}\n")
    
    print(f"Next point to explore maximising EI acqusition function for Standard GP:\n{x_next_ei}\n\
    predicted mean: {mean_ei:.3f}, predicted std: {std_ei:.3f}\n")
    
    print(f"Next point to explore maximising PI acqusition function for Standard GP:\n{x_next_pi}\n\
    predicted mean: {mean_pi:.3f}, predicted std: {std_pi:.3f}\n")

    # Return results as a dictionary
    results = {
        'ucb': {
            'index': idx_ucb,
            'x_next': x_next_ucb,
            'mean': mean_ucb,
            'std': std_ucb,
        },
        'ei': {
            'index': idx_ei,
            'x_next': x_next_ei,
            'mean': mean_ei,
            'std': std_ei,
        },
        'pi': {
            'index': idx_pi,
            'x_next': x_next_pi,
            'mean': mean_pi,
            'std': std_pi,
        }
    }

    return results